In [2]:
import random
import pandas as pd
import numpy as np

In [3]:
def generate_employment_statuses(amount,sample_year):
    SyssStat = []
    for i in range(amount):  
        if sample_year <= 2003:
            status = [1, 5, 6]
            probabilities = [0.6, 0.2, 0.2]
        elif sample_year <= 2011:
            status = [1, 5, 6]
            probabilities = [0.7, 0.2, 0.1]
        else:
            status = [1, 5, 6, 7]
            probabilities = [0.69, 0.2, 0.1, 0.01]
        employ = random.choices(status, probabilities)[0]
        SyssStat.append(employ)

    return SyssStat

def generate_workingtime(SyssStat):
    ArbTid = []
    for s in SyssStat:
        if s == 1 or s == 7:
            status = [1,2,3,4,5,9]
            probabilities = [0.05, 0.1, 0.15, 0.2, 0.4, 0.1]
            time = random.choices(status, probabilities)[0]
            ArbTid.append(time)
        else:
            ArbTid.append(" ")
    return ArbTid 

def generate_job(ArbTid):
    YrkStalln = []
    for i in ArbTid:
        if i == " ":
            YrkStalln.append(" ")
        else:
            status = [0,1,2,4,5]
            probabilities = [0.1, 0.2, 0.49, 0.2, 0.01]
            job = random.choices(status, probabilities)[0]
            YrkStalln.append(job)
    return YrkStalln


def generate_income(ArbTid):
    mean=1000
    std_dev=500
    KU1lnk = []
    KU2lnk = []
    KU3lnk = []

    for i in ArbTid:
        if i == " ":  
            KU1lnk.append(0)
            KU2lnk.append(0)
            KU3lnk.append(0)
        else:
            # Generate incomes from a normal distribution with the specified mean and standard deviation
            ku1 = max(0, int(np.random.normal(mean, std_dev) * 100))
            ku2 = max(0, int(np.random.normal(mean, std_dev) * 100))
            ku3 = max(0, int(np.random.normal(mean, std_dev) * 100))
            
            # Ensure KU1lnk > KU2lnk > KU3lnk
            KU1lnk.append(max(ku1, ku2, ku3))
            generate_ku2 = random.random() < 0.5
            if generate_ku2:
                KU2lnk.append(sorted([ku1, ku2, ku3])[1])
                generate_ku3 = random.random() < 0.1
                if generate_ku3:
                    KU3lnk.append(min(ku1, ku2, ku3))
                else:
                    KU3lnk.append(0)
            else:
                KU2lnk.append(0)
                KU3lnk.append(0)
            

    return KU1lnk, KU2lnk, KU3lnk

def generate_demographic_economic(amount,sample_year):
    SyssStat = generate_employment_statuses(amount,sample_year)
    ArbTid = generate_workingtime(SyssStat)
    YrkStalln = generate_job(ArbTid)
    KU1lnk, KU2lnk, KU3lnk = generate_income(ArbTid)

    employment_data = {
                        'SyssStat'           : SyssStat,
                        'ArbTid'        : ArbTid,
                        'YrkStalln'    : YrkStalln,
                        'KU1lnk'  : KU1lnk,
                        'KU2lnk'     : KU2lnk,
                        'KU3lnk'       : KU3lnk
                        }    
    data = pd.DataFrame.from_dict(employment_data)
    return data

In [4]:
generate_demographic_economic(20,2019)

,SyssStat,ArbTid,YrkStalln,KU1lnk,KU2lnk,KU3lnk
0,1,5,4,163654,157986,0
1,1,3,4,190919,108864,0
2,1,3,2,120051,99806,0
3,6,,,0,0,0
4,1,5,1,181432,0,0
5,1,4,2,194626,0,0
6,1,5,0,189053,115880,0
7,1,5,2,137587,128588,0
8,5,,,0,0,0
9,1,5,1,167319,0,0
